In [11]:
! pip install  xgboost 

  Using cached xgboost-2.1.3-py3-none-win_amd64.whl.metadata (2.1 kB)
   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/124.9 MB 163.8 kB/s eta 0:12:43
   ---------------------------------------- 0.0/124.9 MB 195.7 kB/s eta 0:10:39
   ---------------------------------------- 0.1/124.9 MB 280.5 kB/s eta 0:07:25
   ---------------------------------------- 0.1/124.9 MB 368.6 kB/s eta 0:05:39
   ---------------------------------------- 0.1/124.9 MB 359.9 kB/s eta 0:05:47
   ---------------------------------------- 0.2/124.9 MB 446.5 kB/s eta 0:04:40
   ---------------------------------------- 0.2/124.9 MB 446.5 kB/s eta 0:04:40
   ---------------------------------------- 0.2/124.9 MB 446.5 kB/s eta 0:04:40
   ---------------------------------------- 0.2/124.9 MB 446.5 kB/s eta 0:04:

In [13]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
import xgboost as xgb
from sklearn.metrics import accuracy_score
import numpy as np
import joblib

In [15]:
train_dir = r'E:\FCI\fourth level\computer vision\Alzheimer_s Dataset\train'  
test_dir = r'E:\FCI\fourth level\computer vision\Alzheimer_s Dataset\test'

In [17]:
# Set the input shape
input_shape = (128, 128, 3)
batch_size = 32

In [19]:
# Use ImageDataGenerator for data augmentation
data_generator = ImageDataGenerator(rescale=1./255)

In [21]:
# Load the training images
train_generator = data_generator.flow_from_directory(
    train_dir,
    target_size=(input_shape[0], input_shape[1]),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

Found 5121 images belonging to 4 classes.


In [23]:
# Load the test images
test_generator = data_generator.flow_from_directory(
    test_dir,
    target_size=(input_shape[0], input_shape[1]),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

Found 1279 images belonging to 4 classes.


In [25]:
# Define the model architecture up to the feature extraction layer
model = Sequential([
    Conv2D(32, (3, 3), input_shape=input_shape),
    Activation('relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3)),
    Activation('relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(64),
    Activation('relu'),
    Dropout(0.5)
])

E:\anaconda\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [27]:
# Extract features from training images 
train_features = model.predict(train_generator)
train_features = train_features.reshape(train_features.shape[0], -1)  # Flatten features
train_labels = train_generator.classes

  1/161 ━━━━━━━━━━━━━━━━━━━━ 38s 243ms/step

E:\anaconda\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


161/161 ━━━━━━━━━━━━━━━━━━━━ 15s 95ms/step


In [29]:
# Extract features from test images 
test_features = model.predict(test_generator)
test_features = test_features.reshape(test_features.shape[0], -1)
test_labels = test_generator.classes

40/40 ━━━━━━━━━━━━━━━━━━━━ 2s 52ms/step


In [31]:
# Use XGBoost 
xgb_classifier = xgb.XGBClassifier(
    n_estimators=100,
    learning_rate=0.3,
    max_depth=6,
    eval_metric='mlogloss')
xgb_classifier.fit(train_features, train_labels)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.3, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=6,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=100,
              n_jobs=None, num_parallel_tree=None, objective='multi:softprob', ...)

In [33]:
# Make predictions on the test data
test_predictions = xgb_classifier.predict(test_features)

In [35]:
# Calculate accuracy
test_accuracy = accuracy_score(test_labels, test_predictions)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

Test Accuracy: 55.04%


In [37]:
# Save the Keras CNN model as an .h5 file
model.save('cnn_feature_extractor9.h5')

In [39]:
joblib.dump(xgb_classifier, 'xgb_classifier.pkl')

['xgb_classifier.pkl']

In [71]:
from keras.models import load_model
import joblib
import numpy as np
from tensorflow.keras.preprocessing import image

# Load the saved models
cnn_model = load_model('cnn_feature_extractor9.h5')
xg_classifier = joblib.load('xgb_classifier.pkl')

def classify_image(img_path):
    # Load and preprocess the new image
    img = image.load_img(img_path, target_size=(128, 128))  # Resize to match the CNN input
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array /= 255.0  # Normalize to match the training preprocessing

    # Extract features using the CNN model
    features = cnn_model.predict(img_array)
    features = features.reshape(1, -1)  # Flatten to 1D for xgboost

    # Classify the features with the xgboost model
    prediction = xg_classifier.predict(features)
    
    # Interpret the prediction
    class_labels = {0: 'MildDemented', 1: 'ModerateDemented',2: 'NonDemented',3:'VeryMildDemented'}  # Adjust based on your dataset labels
    result = class_labels[prediction[0]]
    return result

# Test the function with a new image
img_path = r'E:\FCI\fourth level\computer vision\Alzheimer_s Dataset\test\NonDemented\26 (63).jpg'
result = classify_image(img_path)
print(f"The image is classified as: {result}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
The image is classified as: NonDemented
